<a href="https://colab.research.google.com/github/Zamoca42/TIL/blob/main/OS/Mutex_Semaphores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로세스 동기화

- 데이터 정합성 이슈의 발생
  - 동시에 공유 데이터에 접근한다면 데이터 불일치가 발생할 수 있음
  - 데이터 일관성을 유지하려면 협력 프로세스간의 순차적인 실행을 보장하는 매커니즘이 필요

  ↪ Count값을 조정해보자

## 임계구역 문제와 해결안
### 생산자 - 소비자 문제

![스크린샷 2022-11-24 오후 11 20 35](https://user-images.githubusercontent.com/96982072/203823420-3937add4-f89d-4335-8b0e-f364aabad0df.png)

- 무한버퍼
  - 버퍼의 크기에 실질적인 한계가 없음 
- 유한버퍼
  - 버퍼크기고정
    - 버퍼가 비어있으면 ➡ 소비자는 반드시 대기 
    - 모든 버퍼가 채워져 있으면 ➡ 생산자가 대기

1. 공유 메모리를 사용한 생산자 프로세스 (생산될 때마다 count 증가)

  ```
  item next_produced; 

  while (true) {
    /* next_produced에 item 생산 */
    while (count==BUFFER_SIZE); 
    /* 아무일도 안함*/
  
    buffer[in] = next_produced; 
    in = (in + 1) % BUFFER_SIZE;
    count++;
  }
  ```

2. 공유 메모리를 사용한 소비자 프로세스 (소비될 때마다 count 감소)

  ```
  item next_consumed; 
  while (true) {
    while (count==0);
    /* 아무일도 안함 */
    next_consumed = buffer[out]; 
    out = (out + 1) % BUFFER_SIZE;
    
    count--;
    /* next_consumed에 있는 item소비 */
  }
  ```

3. Count 표현 과정

- Count 초기값을 5로 가정

  ```
  count++ 를 풀어서 표현하면, 
    register1 = count
    register1 = register1 +1 
    count = register1
  ```
  
  </br>

  ```
  count-- 를 풀어서 표현하면, 
    register2 = count
    register2 = register2 - 1 
    count = register2
  ```
  
  </br>

  ```
  “count=5”라고 표현되기까지의 과정은,

  T0 : 생산자가 register1 = count 수행 {register1=5}   
  T1 : 생산자가 register1 = register1+1 수행 {register1=6} 
  T2 : 소비자가 register2 = count 수행 {register2=5} 
  T3 : 소비자가 register2 = register2 -1 수행 {register2=4}
  T4 : 생산자가 count = register1 수행 {count=6}
  T5 : 소비자가 count = register2 수행 {count=4}
  ```

- 프로세스가 뒤섞여서 출력

- 경쟁상태

- 임계구역 (Critical-Section)
  - 각 프로세스에는 임계 구역이라고 하는 코드 세그먼트가 존재 
    - 프로세스가 액세스하고, 업데이트하는 데이터, 적어도  
      하나의 다른 프로세스와 공유됨
  - 하나의 프로세스가 임계구역에서 실행 중일 때, 다른 프로세스들은 해당 임계 구역에 접근할 수 없음
  - 일반적으로 프로세스는 진입구역/퇴출구역, 나머지구역으로 구성


## 임계구역 문제 해결을 위한 요구조건

- 상호배제 (Mutual Exclusion)
  - 프로세스 $ P_i $ 가 임계구역에서 실행될 때, 다른 어떤 프로세스도 임계 구역에 접근할 수 없음

- 진행 (Progress)
  - 임계구역에서 실행중인 프로세스가 없고, 임계구역에 들어가고자 하는 프로세스가 존재한다면,  
  나머지 구역에서 실행 중이지 않은 프로세스들만 다음 임계구역에 진입할 프로세스를 결정하는데 참여할 수 있음.  
  이 선택은 무한정 연기할 수 없음

- 제한이 있는 대기시간 (Bounded Waiting)
  - 프로세스가 자신의 임계구역에 진입하려는 요청을 한 후부터,  
  그 요청이 허용될 때까지 다른 프로세스들이 자신들의 임계구역에 진입하도록 허용되는 횟수에 한계가 있어야 함

## 피터슨의 해결안 - 해결방법

- 두 개의 프로세스가 두 개의 데이터 항목을 공유하게 하도록 하여 해결

- int turn
  - 임계구역으로 진입할 순번을 표시

- Boolean flag[2]
  - 프로세스가 임계구역으로 진입할 준비가 되었다는 여부를 표시
  - if) flag[i] = true ➡ Pi 는 임계구역으로 진입할 준비가 되었다는 뜻

## 피터슨의 해결안 – 코드로 보는 해결방법

![스크린샷 2022-11-24 오후 11 59 08](https://user-images.githubusercontent.com/96982072/203823587-52e02e15-3ffc-43c0-910c-2562cb1c4eab.png)

- 상호배제
- 진행
- 제한이 있는 대기시간

## 뮤텍스 락

![스크린샷 2022-11-25 오전 12 01 18](https://user-images.githubusercontent.com/96982072/203823823-a9340407-0478-429c-9cb0-d40a3533e884.png)


- 피터슨의 해결안의 한계점, 하드웨어 기반 해결책의 구현 난이도의 문제(응용프로그래머 사용불가)
  - 임계구역 문제를 해결하기 위한 소프트웨어 도구 개발

- 뮤텍스 락을 이용하여 임계영역을 보호, 경쟁 방지
- 프로세스는 임계영역에 진입하기 전 반드시 락을 획득해야함. 임계 영역 사용 종료 시 잠금 해제
- 잠금 사용 가능 여부를 나타내는 Boolean 변수 사용
- 그러나, busy waiting이 존재함(spinlock)

## 세마포어의 종류와 사용법

- 세마포어(Semaphores)
  - 정수 변수(S)
  - wait() - P, signal() - V연산으로만 접근이 가능

    ```
    wait (S){
      while (S <= 0); /* busy wait*/ 
      S--;
    }
    signal (S){
      S++; 
    }
    ```

- 뮤텍스보다 더 정교하게 동기화 할 수 있는 방법이 필요해짐
- 카운팅 세마포어 
  - 영역(domain)에 제한 없음, 유한한 개수를 가진 자원에 대한 접근을 제어하는데 사용
- 이진 세마포어
  - 0과1사이 값만 가능.뮤텍스락과 유사하게 동작
- 각 자원을 사용하려는 프로세스는 세마포어에 wait연산 수행(S감소)

- 프로세스가 자원을 방출할때는 signal연산 수행(S증가)

- 다양한 동기화 문제를 해결하기 위해 세마포어 사용 가능

  - ex) P1은 S1 명령문을, P2는 S2명령문을 병행하려고 하는 두 프로세스가 존재할 때,  
  S2는 S1이 끝난 뒤에만 수행되어야 한다고 가정

## 세마포어 정리

- 세마포어 역시, busy waiting이라는 단점이 존재

- busy waiting 대신, 자신을 봉쇄하여 단점 상쇄
  - Sleep(일시중단)
    - 프로세스를 해당 세마포어와 연결된 대기큐에 위치시킴 
  - Wakeup
    - 프로세스를 대기 상태에서 준비 상태로 변경

## 교착상태(Deadlock)와 기아상태(Starvation)

![스크린샷 2022-11-25 오전 12 32 49](https://user-images.githubusercontent.com/96982072/203823951-5cde4841-1462-4a19-b334-ce1ab5f991d8.png)


- 어떻게 교착 상태를 해결?
  - 한 대의 차량이 후진하면 해결 가능(선점 리소스 및 롤백)
- 교착상태 발생 시 여러대의 차량을 후진시켜야 할 수도 있음 
- 기아 발생 가능성 있음

## 교착상태(Deadlock)

![스크린샷 2022-11-25 오전 12 34 07](https://user-images.githubusercontent.com/96982072/203824053-0fe7f9fc-66ef-4e6d-bd3f-917337208f9f.png)

- P0이 wait(S)를 실행, P1이 wait(Q)를 실행한다고 가정
- P0이 wait(Q)를 실행할 때, P0은 P1이 signal(Q)를 실행할 때까지 기다려야 함
- 마찬가지로, P1이 wait(S)를 실행할 때는 P0이 signal(S)를 실행할 때까지 기다려야 함

  ↪ 이들 시그널 연산들은 실행될 수 없기 때문에 P0과 P1은 교착상태가 됨

## 교착상태의 특징

- 한 시스템에 다음 네 가지 조건 동시성립 시 교착상태가 발생할 수 있음

- 상호배제 (Mutual exclusion)
  - 두 프로세스는 동시에 같은 자원에 접근할 수 없음
  - 다른 프로세스가 그 자원을 요청하면, 요청프로세스는 자원이 해제 될 때까지 대기한 뒤 사용 가능

  ➡ 공유 가능한 리소스 설정(ex. 읽기 전용 파일)

- 점유하며 대기 (Hold-and-wait)
  - 프로세스는 최소한 하나의 자원을 점유한 채, 현재 다른 프로세스에 의해 점유된 자원을 추가로 얻기 위해 반드시 대기해야 함

  ➡ 프로세스가 작업을 수행하기 전에 필요한 자원을 모두 요청하고 획득해야함(최대 자원 할당)

  ➡ 단점 : 리소스 활용도 낮음 / 기아 발생 가능성


- 비선점 (No preemption)
  - 자원들을 선점할 수 없어야 함
  - 자원이 강제적으로 해제될 수 없고, 점유하고 있는 프로세스가 태스크를 종료한 후에만 해제됨

  ➡ 이미 할당된 자원에 선점권이 없어야 함

  ➡ 기존 사용중이던 프로세스가 작업 내용을 잃을 수 있음

- 순환 대기 (Circular wait)
  - 자원들이 cyclic하게 점유한 자원들을 대기해야 함

  ➡ 리소스에 고유한 번호를 할당. 번호 순서대로 리소스를 요청하도록 함

  ➡ 작업에 필요한 자원은 오래 전부터 할당 받고 있어야 하므로 자원 낭비 가능성

